In [19]:
import os
import re
import cantera as ct

directory = '/work/westgroup/nora/Code/projects/halogens/refrigerants/blends'

def convert_files():
    """Proved with a list of blend files, 
    specify the number of fuel species in the blend (num), this will run flame speed calc on all of them.
    
    example: make_blend([list using os.listdir(directory)])
    """
    
    converter = \
    '''

import cantera as ct
import numpy as np
import pandas as pd
import os 
import re
from subprocess import getoutput
import sys
import csv


print("Running Cantera Version: " + str(ct.__version__))





############### copies chemkin files to dups folder #############################


directory ='.'


#os.system('source activate ct_env') #if on local, this is cantera 2.6 beta
os.system('source activate cantera_env') #if on discovery (will be cantera 2.5)


#copy folders so i dont screw up the original, and change into the new directory with copies
os.makedirs('dups', exist_ok=True)

#copy chem.inp file into dups folder, and will then convert this copy into .cti
os.command = f'scp chem.inp dups/dup_chem.inp'
os.system(os.command)
os.system('scp tran.dat dups/tran.dat')

#now look in the dups folder
os.chdir('./dups')









############################# converts the dup_chem.inp files to .cti files #############################

x = 0
while x == 0: 
#this is a string of the output when I try to convert this file to a cti file. Will probably produce an error
    output = getoutput( f'ck2cti --input=dup_chem.inp --transport=tran.dat') 
    #if command passed without an error
    if re.search('PASSED',output):
        print('**************************command passed, converting to cti***************************')
        x += 1 
    #if command generated the Duplicate error
    else:
        print('*******************************needs some work****************************************')
        if re.search('Encountered\sunmarked\sduplicate\sreaction',output):
            match = re.search('See\slines\s([0-9]+)\sand\s([0-9]+)\sof\sthe\sinput\sfile',output)
            #capture the line numbers with the duplicate reactions
            line_numbers = [int(match.group(1)), int(match.group(2))]
            print(f'Unmarked duplicates on lines {match.group(1)} and {match.group(2)}')
            print('Editing chemkin file to allow conversion to .cti')
            #write the lines of the chemkin input file to a list so that I can insert the "DUPLICATE" statement
            with open('dup_chem.inp','r') as f:
                data = f.readlines()
                print(data[line_numbers[0]-1], data[line_numbers[1]-1])

            #start editing the .inp file below

            #'adjustments' will make sure that, even when I add an element in 'data', my index will still be correct
            adjustments = [0,1]
            for i,adjust in zip(line_numbers,adjustments): 
                start = i+adjust-1
                count = 0 
                while count == 0: 
                    #if you don't see a blank line after the duplicated reaction line, keep going until you do
                    if not re.search('^\\n', data[start]): 
                        print('no match')
                        print(start)
                        print(data[start])
                        start += 1
                    #when we get to the blank line after the reaction block, insert "DUPLICATE" and stop the loop for this line number
                    else: 
                        print('there is a match')
                        data.insert(start,'DUPLICATE')
                        count = 1 
            #now overwrite the input file with the change 
            with open(f'dup_chem.inp','w+') as f: 
                for l in data: 
                    f.write(l)
                    x==0

        #if the command generated an error that is not the Duplicate error
        else:
            #if code ever gets to here, just cry
            print('There is another error, see Output')
            print(output)
            x += 1
            
            
'''   
    directory = '/work/westgroup/nora/Code/projects/halogens/refrigerants/blends'
    os.chdir(directory)

    list_of_blends = [file for file in os.listdir(directory) if re.match('[A-Z0-9]+_[A-Z0-9]+', file)]

    for blend in list_of_blends: 
        d = os.path.join(directory, f'{blend}/chemkin')
        with open(os.path.join(d,'converter.py'),'w') as f:
            for l in converter:
                f.write(l)
        print('rewrote it!')

directory = '/work/westgroup/nora/Code/projects/halogens/refrigerants/blends'

list_of_blends = [file for file in os.listdir(directory) if re.match('[A-Z0-9]+_[A-Z0-9]+', file)]
#list_of_blends = ['CH3F_CH3CF3']

convert_files()


            

rewrote it!
rewrote it!
rewrote it!
rewrote it!
rewrote it!
rewrote it!
rewrote it!
rewrote it!
rewrote it!
rewrote it!
rewrote it!
rewrote it!
rewrote it!
rewrote it!
rewrote it!
rewrote it!
rewrote it!
rewrote it!
rewrote it!
rewrote it!
rewrote it!


In [1]:
import os
import re
import cantera as ct

def make_flame_script(list_of_blend_files, num):
 
    species = {'CH2F2': [0.11,0.225,0.174] ,'CH3F': [0.08,.225,0.123],'C2H5F': [0.025,0.150,0.0654],'CH2FCH2F' :[0.0375,0.150,0.0775],'CH2FCHF2' :[0.075,0.150,0.095],'CH3CF3' :[0.07,0.150,0.095],'CH3CHF2':[0.0375, 0.150,0.0775]}

    for file in list_of_blend_files: 


        script = '''
        ############################# calculates flamespeeds for the .cti files at different equivalence ratios. Uses initial guess from the previous model #############################

import cantera as ct
import numpy as np
import pandas as pd
import os
import csv 

print("Running Cantera Version: " + str(ct.__version__))


To = 298
Po = 1e5 # ct.one_atm
vol_frac_list = np.arange(0.025, 0.25, step=0.01)

cti_files = ['dup_chem.cti']

for file in cti_files: 
    #make directory to store flame speed calculations, and a header for csv files
    d = f'./flame_calcs/chem_CALC'
    os.makedirs(d, exist_ok=True)
    gas = ct.Solution(f'./chemkin/dups/{file}')
    results = {}    
    #this part calculates the flame calcs for all of the vol_frac list, using the previous vol frac count as a guess for the current one
    for i in  range(len(vol_frac_list)):
        try:

            string = f'****************************starting new volume fraction: {vol_frac_list[i]}**************************'
            print(string)

            x = vol_frac_list[i]
            norm_ox = (1-x)*.21

        '''

        script_2 = \
        '''
            gas.TPX = To, Po, vol_frac_dict
            width = 0.08
            flame = ct.FreeFlame(gas, width=width)
            flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1) 
            flame.max_time_step_count = 900
            loglevel = 1 
            if i!=0:
                d = f'./flame_calcs/chem_CALC/test_{vol_frac_list[i-1]}.csv'
                if os.path.exists(d):  
                    arr2 = ct.SolutionArray(gas)
                    arr2.read_csv(d)
                    flame.set_initial_guess(data=arr2)
                    print(' initial guess has been set')
            #"False" stops the calculation from retrying over and over, thanks Chao 
            flame.solve(loglevel=loglevel, auto=False)
            #flame.solve(loglevel=loglevel, auto=True)
            Su = flame.velocity[0]
            results[x] = Su
            sltn = flame.to_solution_array()
            df1 = sltn.to_pandas()
            #edited this here!! index=False
            df1.to_csv(f'./flame_calcs/chem_CALC/test_{x}.csv', index=False)


        except Exception as e: 
            print(f'********************passed volume fraction:{vol_frac_list[i]}, error: {e}*************************************')
            pass

    vol_fracs = list(results.keys())
    flame_speeds = list(results.values())


    print("volume fractions are:")
    print(vol_fracs)

    print("flame speeds are:")
    print(flame_speeds)


    with open('final_calcs.csv', 'w+') as g:
        writers = csv.writer(g)
        writers.writerow(vol_fracs)
        writers.writerow(flame_speeds)


        '''




 #       species = {'CH2F2': [0.11,0.225,0.174] ,'CH3F': [0.08,.225,0.123],'C2H5F': [0.025,0.150,0.0654],'CH2FCH2F' :[0.0375,0.150,0.0775],'CH2FCHF2' :[0.075,0.150,0.095],'CH3CF3' :[0.07,0.150,0.095],'CH3CHF2':[0.0375, 0.150,0.0775]}

 #       for file in list_of_blend_files: 

        #make pattern that re.match will use 
        pattern = ''
        l = list(range(num-1))
        print(l)
        for i in l:
            pattern += '([A-Z0-9]+)_'
        pattern += '([A-Z0-9]+)'
        print(pattern)

        #match and make a list of the species matched in each file
        match = re.match(pattern, file)
        label = []
        l = list(range(num))
        for i in l: 
            label.append(match.group(i+1))

        #correct label if it has a funky name
        labels_we_hate={'CH2FCHF2':'C2H3F3','CH2FCH2F':'C2H4F2'}
        for i in range(len(label)):
            if label[i] in labels_we_hate.keys(): 
                label[i]= labels_we_hate[label[i]]
        print(label)

        #add oxygen to list, and rename 
        ox_label = 'O2' 
        species_to_look_for = [i for i in label]
        species_to_look_for.append(ox_label)
        print(species_to_look_for)

        #get species name from cti file

        chem_file = f'./{file}/chemkin/dups/dup_chem.cti'
        gas = ct.Solution(chem_file)
        species_names_we_want = {}
        for i in species_to_look_for:
            for name in gas.species_names:
                pattern = f"{i}\([0-9]+\)"
                m = re.match(pattern, name)
                if m:
                    species_names_we_want[i]=name
                    print(i, name)
                    break
                else:
                    print(f"didnt find {i}")
        print(file, species_names_we_want)



        #now make the damn thing
        script += make_vol_frac_dict(list(species_names_we_want.values()),num)
        script += script_2

        #directory = '/work/westgroup/nora/Code/projects/halogens/refrigerants/blends'
        #os.chdir(directory)

        #list_of_blends = [file for file in os.listdir(directory) if re.match('[A-Z0-9]+_[A-Z0-9]+', file)]

        #for blend in list_of_blends: 
        d = os.path.join(directory, file)
        print(script)
        print('#######################################################')
        with open(os.path.join(d,'flame_calc.py'),'w') as f:
            for l in script:
                f.write(l)
        print('rewrote it!')



In [2]:
def make_vol_frac_dict(label_list,num):
    '''Makes a volume fraction dictionary, assuming there are equal parts fuel in the blend'''

    vol_frac_dict = \
    '''
            vol_frac_dict = {'''
    l = list(range(len(label_list)-1))
    for i in l:
        vol_frac_dict += f"'{label_list[i]}': (x/{num}/norm_ox), "
    vol_frac_dict += f"'{label_list[-1]}':((1-x)*.21)/norm_ox, 'N2':((1-x)*0.79)/norm_ox"
    vol_frac_dict += '}'
    

    print(vol_frac_dict)
    return vol_frac_dict


In [3]:
directory = '/work/westgroup/nora/Code/projects/halogens/refrigerants/blends'
os.chdir(directory)
list_of_blends = [file for file in os.listdir(directory) if re.match('[A-Z0-9]+_[A-Z0-9]+', file)]

make_flame_script(list_of_blends,2)

[0]
([A-Z0-9]+)_([A-Z0-9]+)
['C2H5F', 'CH2F2']
['C2H5F', 'CH2F2', 'O2']
didnt find C2H5F
didnt find C2H5F
C2H5F C2H5F(1)
didnt find CH2F2
didnt find CH2F2
didnt find CH2F2
CH2F2 CH2F2(2)
didnt find O2
didnt find O2
didnt find O2
didnt find O2
O2 O2(3)
C2H5F_CH2F2 {'C2H5F': 'C2H5F(1)', 'CH2F2': 'CH2F2(2)', 'O2': 'O2(3)'}

            vol_frac_dict = {'C2H5F(1)': (x/2/norm_ox), 'CH2F2(2)': (x/2/norm_ox), 'O2(3)':((1-x)*.21)/norm_ox, 'N2':((1-x)*0.79)/norm_ox}

        ############################# calculates flamespeeds for the .cti files at different equivalence ratios. Uses initial guess from the previous model #############################

import cantera as ct
import numpy as np
import pandas as pd
import os
import csv 

print("Running Cantera Version: " + str(ct.__version__))


To = 298
Po = 1e5 # ct.one_atm
vol_frac_list = np.arange(0.025, 0.25, step=0.01)

cti_files = ['dup_chem.cti']

for file in cti_files: 
    #make directory to store flame speed calculations, and a header for 

didnt find C2H5F
didnt find C2H5F
C2H5F C2H5F(1)
didnt find CH3CHF2
didnt find CH3CHF2
didnt find CH3CHF2
CH3CHF2 CH3CHF2(2)
didnt find O2
didnt find O2
didnt find O2
didnt find O2
O2 O2(3)
C2H5F_CH3CHF2 {'C2H5F': 'C2H5F(1)', 'CH3CHF2': 'CH3CHF2(2)', 'O2': 'O2(3)'}

            vol_frac_dict = {'C2H5F(1)': (x/2/norm_ox), 'CH3CHF2(2)': (x/2/norm_ox), 'O2(3)':((1-x)*.21)/norm_ox, 'N2':((1-x)*0.79)/norm_ox}

        ############################# calculates flamespeeds for the .cti files at different equivalence ratios. Uses initial guess from the previous model #############################

import cantera as ct
import numpy as np
import pandas as pd
import os
import csv 

print("Running Cantera Version: " + str(ct.__version__))


To = 298
Po = 1e5 # ct.one_atm
vol_frac_list = np.arange(0.025, 0.25, step=0.01)

cti_files = ['dup_chem.cti']

for file in cti_files: 
    #make directory to store flame speed calculations, and a header for csv files
    d = f'./flame_calcs/chem_CALC'
    os.ma

didnt find CH2F2
didnt find CH2F2
CH2F2 CH2F2(1)
didnt find CH3F
didnt find CH3F
didnt find CH3F
CH3F CH3F(2)
didnt find O2
didnt find O2
didnt find O2
didnt find O2
O2 O2(3)
CH2F2_CH3F {'CH2F2': 'CH2F2(1)', 'CH3F': 'CH3F(2)', 'O2': 'O2(3)'}

            vol_frac_dict = {'CH2F2(1)': (x/2/norm_ox), 'CH3F(2)': (x/2/norm_ox), 'O2(3)':((1-x)*.21)/norm_ox, 'N2':((1-x)*0.79)/norm_ox}

        ############################# calculates flamespeeds for the .cti files at different equivalence ratios. Uses initial guess from the previous model #############################

import cantera as ct
import numpy as np
import pandas as pd
import os
import csv 

print("Running Cantera Version: " + str(ct.__version__))


To = 298
Po = 1e5 # ct.one_atm
vol_frac_list = np.arange(0.025, 0.25, step=0.01)

cti_files = ['dup_chem.cti']

for file in cti_files: 
    #make directory to store flame speed calculations, and a header for csv files
    d = f'./flame_calcs/chem_CALC'
    os.makedirs(d, exist_ok=True)
  

didnt find C2H4F2
didnt find C2H4F2
C2H4F2 C2H4F2(1)
didnt find CH3CHF2
didnt find CH3CHF2
didnt find CH3CHF2
CH3CHF2 CH3CHF2(2)
didnt find O2
didnt find O2
didnt find O2
didnt find O2
O2 O2(3)
CH2FCH2F_CH3CHF2 {'C2H4F2': 'C2H4F2(1)', 'CH3CHF2': 'CH3CHF2(2)', 'O2': 'O2(3)'}

            vol_frac_dict = {'C2H4F2(1)': (x/2/norm_ox), 'CH3CHF2(2)': (x/2/norm_ox), 'O2(3)':((1-x)*.21)/norm_ox, 'N2':((1-x)*0.79)/norm_ox}

        ############################# calculates flamespeeds for the .cti files at different equivalence ratios. Uses initial guess from the previous model #############################

import cantera as ct
import numpy as np
import pandas as pd
import os
import csv 

print("Running Cantera Version: " + str(ct.__version__))


To = 298
Po = 1e5 # ct.one_atm
vol_frac_list = np.arange(0.025, 0.25, step=0.01)

cti_files = ['dup_chem.cti']

for file in cti_files: 
    #make directory to store flame speed calculations, and a header for csv files
    d = f'./flame_calcs/chem_CALC'

didnt find C2H3F3
didnt find C2H3F3
C2H3F3 C2H3F3(1)
didnt find CH3F
didnt find CH3F
didnt find CH3F
CH3F CH3F(2)
didnt find O2
didnt find O2
didnt find O2
didnt find O2
O2 O2(3)
CH2FCHF2_CH3F {'C2H3F3': 'C2H3F3(1)', 'CH3F': 'CH3F(2)', 'O2': 'O2(3)'}

            vol_frac_dict = {'C2H3F3(1)': (x/2/norm_ox), 'CH3F(2)': (x/2/norm_ox), 'O2(3)':((1-x)*.21)/norm_ox, 'N2':((1-x)*0.79)/norm_ox}

        ############################# calculates flamespeeds for the .cti files at different equivalence ratios. Uses initial guess from the previous model #############################

import cantera as ct
import numpy as np
import pandas as pd
import os
import csv 

print("Running Cantera Version: " + str(ct.__version__))


To = 298
Po = 1e5 # ct.one_atm
vol_frac_list = np.arange(0.025, 0.25, step=0.01)

cti_files = ['dup_chem.cti']

for file in cti_files: 
    #make directory to store flame speed calculations, and a header for csv files
    d = f'./flame_calcs/chem_CALC'
    os.makedirs(d, exist_o

didnt find CH3CF3
didnt find CH3CF3
CH3CF3 CH3CF3(1)
didnt find C2H3F3
didnt find C2H3F3
didnt find C2H3F3
C2H3F3 C2H3F3(2)
didnt find O2
didnt find O2
didnt find O2
didnt find O2
O2 O2(3)
CH3CF3_CH2FCHF2 {'CH3CF3': 'CH3CF3(1)', 'C2H3F3': 'C2H3F3(2)', 'O2': 'O2(3)'}

            vol_frac_dict = {'CH3CF3(1)': (x/2/norm_ox), 'C2H3F3(2)': (x/2/norm_ox), 'O2(3)':((1-x)*.21)/norm_ox, 'N2':((1-x)*0.79)/norm_ox}

        ############################# calculates flamespeeds for the .cti files at different equivalence ratios. Uses initial guess from the previous model #############################

import cantera as ct
import numpy as np
import pandas as pd
import os
import csv 

print("Running Cantera Version: " + str(ct.__version__))


To = 298
Po = 1e5 # ct.one_atm
vol_frac_list = np.arange(0.025, 0.25, step=0.01)

cti_files = ['dup_chem.cti']

for file in cti_files: 
    #make directory to store flame speed calculations, and a header for csv files
    d = f'./flame_calcs/chem_CALC'
    os.m

didnt find CH3CHF2
didnt find CH3CHF2
CH3CHF2 CH3CHF2(1)
didnt find C2H3F3
didnt find C2H3F3
didnt find C2H3F3
C2H3F3 C2H3F3(2)
didnt find O2
didnt find O2
didnt find O2
didnt find O2
O2 O2(3)
CH3CHF2_CH2FCHF2 {'CH3CHF2': 'CH3CHF2(1)', 'C2H3F3': 'C2H3F3(2)', 'O2': 'O2(3)'}

            vol_frac_dict = {'CH3CHF2(1)': (x/2/norm_ox), 'C2H3F3(2)': (x/2/norm_ox), 'O2(3)':((1-x)*.21)/norm_ox, 'N2':((1-x)*0.79)/norm_ox}

        ############################# calculates flamespeeds for the .cti files at different equivalence ratios. Uses initial guess from the previous model #############################

import cantera as ct
import numpy as np
import pandas as pd
import os
import csv 

print("Running Cantera Version: " + str(ct.__version__))


To = 298
Po = 1e5 # ct.one_atm
vol_frac_list = np.arange(0.025, 0.25, step=0.01)

cti_files = ['dup_chem.cti']

for file in cti_files: 
    #make directory to store flame speed calculations, and a header for csv files
    d = f'./flame_calcs/chem_CALC'


NameError: name 'species_names_we_want' is not defined

In [19]:
=


NameError: name 'vol_frac_dict' is not defined

In [22]:
''' remove any files that you create by mistake '''



directory = '/work/westgroup/nora/Code/projects/halogens/refrigerants/blends'

list_of_blends = [file for file in os.listdir(directory) if re.match('[A-Z0-9]+_[A-Z0-9]+', file)]

for blend in list_of_blends: 
    d = os.path.join(directory, f'{blend}/chemkin')
    cwd = os.getcwd()
    print(cwd)
    os.chdir(d)
    cwd = os.getcwd()
    print(cwd)
    command = 'rm converter'
    os.system(command)


/work/westgroup/nora/Code/projects/halogens/refrigerants/blends
/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/C2H5F_CH2F2/chemkin
/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/C2H5F_CH2F2/chemkin
/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/C2H5F_CH2FCH2F/chemkin
/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/C2H5F_CH2FCH2F/chemkin
/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/C2H5F_CH2FCHF2/chemkin
/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/C2H5F_CH2FCHF2/chemkin
/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/C2H5F_CH3CHF2/chemkin
/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/C2H5F_CH3CHF2/chemkin
/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/C2H5F_CH3F/chemkin
/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/C2H5F_CH3F/chemkin
/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/CH2F2_CH3CF3/chemk

In [35]:

cwd = os.getcwd()
print(cwd)

/work/westgroup/nora/Code/projects/halogens/refrigerants/blends
